In [9]:
!pip3 install langchain==0.3.1
!pip3 install langchain-openai==0.2.1
!pip3 install langchain-community==0.3.1
!pip3 install streamlit
!pip3 install openai
!pip3 install tiktoken
!pip3 install chromadb
!pip3 install pypdf
!pip3 install langchain-core==0.3.7
!pip3 install PyMuPDF
!pip3 install pdfplumber

166.47s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached langchain-0.3.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
Using cached langchain-0.3.1-py3-none-any.whl (1.0 MB)
Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


180.24s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached langchain_openai-0.2.1-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain_openai-0.2.1-py3-none-any.whl (49 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


186.65s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


...


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


213.10s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip


UnboundLocalError: cannot access local variable 'child' where it is not associated with a value

219.08s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [10]:
from dotenv import load_dotenv
load_dotenv()
import openai
import os
openai.api_key = os.environ.get("OPENAI_API_KEY")

import streamlit as st
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pprint import pprint
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
import json
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
import logging
import time

# 문제 진단: 프로그램이 복잡할수록 버그나 오류를 찾기 어려워지는데, 로그는 코드 실행 흐름을 기록하여 문제 발생 지점을 쉽게 찾아낼 수 있게 도와줍니다.
# 로그 설정: 기본 로그 설정을 초기화합니다.
# 특정 로거 레벨 설정: "langchain.retrievers.multi_query" 로거의 로그 레벨을 INFO로 설정하여 해당 모듈의 로그를 표시합니다.
# 파일에 로그 저장: 로그 메시지를 'info.log' 파일에 시간, 로거 이름, 로그 레벨, 메시지 형식으로 저장합니다. 'a' 모드는 기존 파일에 내용을 추가합니다.
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)
logging.basicConfig(filename='info.log', level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', filemode='a')

# 사용할 챗봇 LLM 모델 설정해주기
# temperature는 0에 가까워질수록 전달해준 설명 및 질문 내에서만 답변을 내뱉고, 1에 가까워질수록 창의적인 답변을 내뱉음
chat = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.1)

# 챗봇에게 질문하고 답변 출력
#response = chat.invoke("오늘 날짜가 뭐야? ")
#print(response.content)


import requests
from bs4 import BeautifulSoup

# Function to extract iframe content
def extract_iframe_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all iframes on the page
    iframes = soup.find_all('iframe')
    
    iframe_data = []
    for iframe in iframes:
        # Extract the src attribute (URL of the iframe content)
        iframe_src = iframe.get('src')
        if iframe_src:
            # Make a request to the iframe URL
            try:
                iframe_response = requests.get(iframe_src)
                if iframe_response.status_code == 200:
                    # Parse and store the iframe's HTML content
                    iframe_soup = BeautifulSoup(iframe_response.content, 'html.parser')
                    iframe_data.append(iframe_soup.get_text())
            except Exception as e:
                print(f"Error fetching iframe {iframe_src}: {e}")
    
    return '\n'.join(iframe_data)

# URLs to scrape
urls = [
    "https://skt-basic-ui-component.webflow.io/"
]

# Combine main page data with iframe data
all_data = []
for url in urls:
    # Load main page data using WebBaseLoader
    loader = WebBaseLoader(url)
    data = loader.load()
    
    # Extract data from iframes on the page
    iframe_content = extract_iframe_content(url)
    
    # Add both main page and iframe data to all_data list
    all_data.extend(data)  # Main page data
    all_data.append(iframe_content)  # Iframe data

data = all_data


# 1) WEB 데이터 로드하기
#loader = WebBaseLoader("https://en.wikipedia.org/wiki/Elon_Musk")
#loader = WebBaseLoader("https://skt-basic-ui-component.webflow.io/")
#data = loader.load()

# 2) 데이터 쪼개기
# chunk_size는 쪼갤 텍스트의 사이즈를 뜻합니다.
# 쪼개는 이유는 context window가 제한되어 있기 때문.
# 만약 더 큰 데이터 파일이고 chunk_size가 높다면, chunk_overlap = 200으로 설정
# 각 청크 사이에 중요한 맥락을 놓치지 않도록 어느정도 중복되게 하는 것
chunk_size = 1000
chunk_overlap = 200

# RecursiveCharacterTextSplitter: 문서를 \n과 같은 일반적인 구분자를 사용하여 재귀적으로 분할하여 각 청크가 적절한 크기가 될 때까지 분할합니다.
# chunk_overlap은 전 문장의 의미를 어느정도 연결할 수 있도록 하는 역할을 합니다. 보통 10 ~ 20% 정도의 chunk를 할당합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
all_splits = text_splitter.split_documents(data)

pprint(all_splits)

USER_AGENT environment variable not set, consider setting it to identify your requests.


AttributeError: 'str' object has no attribute 'page_content'

In [ ]:

import requests
from bs4 import BeautifulSoup

# Function to extract iframe content
def extract_iframe_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all iframes on the page
    iframes = soup.find_all('iframe')
    
    iframe_data = []
    for iframe in iframes:
        # Extract the src attribute (URL of the iframe content)
        iframe_src = iframe.get('src')
        if iframe_src:
            # Make a request to the iframe URL
            try:
                iframe_response = requests.get(iframe_src)
                if iframe_response.status_code == 200:
                    # Parse and store the iframe's HTML content
                    iframe_soup = BeautifulSoup(iframe_response.content, 'html.parser')
                    iframe_data.append(iframe_soup.get_text())
            except Exception as e:
                print(f"Error fetching iframe {iframe_src}: {e}")
    
    return '\n'.join(iframe_data)

# URLs to scrape
urls = [
    "https://skt-basic-ui-component.webflow.io/"
]

# Combine main page data with iframe data
all_data = []
for url in urls:
    # Load main page data using WebBaseLoader
    loader = WebBaseLoader(url)
    data = loader.load()
    
    # Extract data from iframes on the page
    iframe_content = extract_iframe_content(url)
    
    # Add both main page and iframe data to all_data list
    all_data.extend(data)  # Main page data
    all_data.append(iframe_content)  # Iframe data

data = all_data

